# **Are You the Asshole? Judging r/AITA Drama with NLP and ML**
*Final Project for CMSC176: Natural Language Processing* </br> </br>
**Authors**: 
- Igot, Leanne Gabrielle 
- Lahaylahay, Dunn Dexter
- Molanda, Clelia 
- Peladas, Daenielle Rai </br>
</br>


**Last Modified**: December 8, 2024

## **Data Acquisition**

In [2]:
import os
import re
import sys
import json
import pandas as pd

In [3]:
os.getcwd()

'/mnt/c/Users/gloss/Documents/5th Year (2024 - 25)/1st Sem (5)/CMSC 176 A/Final Project/AITAH-FAFO/notebooks'

This function generates a DataFrame from all the scraped comments stored in the specified directory.

In [6]:
def create_dataframe_from_comments():
    # Directory path where the JSON files are stored
    comments_dir = os.path.join('/mnt/c/Users/gloss/Documents/5th Year (2024 - 25)/1st Sem (5)/CMSC 176 A/Final Project/AITAH-FAFO/scrapes/all-comments') # Change this to your current working directory

    # Initialize a list to hold the data
    data = []

    # Go through all files in the directory
    for filename in os.listdir(comments_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(comments_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                # Load the JSON file
                content = json.load(file)

                metadata = content['data']['submission_metadata']
                comments = content['data']['comments']

                # Extract the required fields
                title = metadata['title']
                selftext = metadata['selftext']
                aitah_tag = count_YTA_NTA(comments)

                # Append to the data list
                data.append({'title': title, 'selftext': selftext, 'aitah_tag': aitah_tag})
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    return df

This function analyzes the comment section to count occurrences of keywords related to 'You're the Asshole' (YTA) and 'Not the Asshole' (NTA). Based on the keyword counts, it determines the overall judgment of whether the subject is considered the asshole or not.

In [7]:
def count_YTA_NTA(comments_list):
    # Extract 'body' content from each comment
    bodies = [comment['body'] for comment in comments_list if 'body' in comment]

    # Initialize counters for YTA and NTA
    YTA = 0
    NTA = 0

    # Loop through each comment body and count occurrences of YTA and NTA (including YTAH and NTAH)
    for body in bodies:
        YTA += len(re.findall(r'\bYTA\b', body)) + len(re.findall(r'\bYTAH\b', body))
        NTA += len(re.findall(r'\bNTA\b', body)) + len(re.findall(r'\bNTAH\b', body))

    if YTA > NTA:
        return 1
    return 0

In [8]:
# Specify the root directory
root_directory = 'AITAH-FAFO'
df = create_dataframe_from_comments()

In [9]:
df.head()

,title,selftext,aitah_tag
0,AITA for not inviting my mom to my wedding be...,I (28M) am getting married in a few months to ...,0
1,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0
2,AITA -,throw away account so no one sees..\n\nMy \[24...,0
3,AITA - my 20 year old flatmate asks me questio...,For context I’m 33; and my flatmate is 20.\nSh...,0
4,"AITA - my husband never let's me sleep in, so ...","As the title states, I wake my husband up ever...",0


### **Basic Insights**

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      574 non-null    object
 1   selftext   574 non-null    object
 2   aitah_tag  574 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.6+ KB


## **Data Wrangling**

In [11]:
# Duplicating raw dataset for FastText processing
df2 = df.copy()

In [12]:
df.isnull().sum()

title        0
selftext     0
aitah_tag    0
dtype: int64

There are no null values shown here. However, there were some posts that were deleted and therefore must be removed since the text in the post itself is necessary for analysis.

In [13]:
df = df[df['selftext'] != '[deleted]']
df = df[df['selftext'] != '[removed]']

In [14]:
pd.set_option('display.max_rows', None)
df.head()

,title,selftext,aitah_tag
0,AITA for not inviting my mom to my wedding be...,I (28M) am getting married in a few months to ...,0
1,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0
2,AITA -,throw away account so no one sees..\n\nMy \[24...,0
3,AITA - my 20 year old flatmate asks me questio...,For context I’m 33; and my flatmate is 20.\nSh...,0
4,"AITA - my husband never let's me sleep in, so ...","As the title states, I wake my husband up ever...",0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 547 entries, 0 to 573
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      547 non-null    object
 1   selftext   547 non-null    object
 2   aitah_tag  547 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 17.1+ KB


In [ ]:
# Balance dataset
import pandas as pd

# First, check the counts of each class
class_counts = df['aitah_tag'].value_counts()
print(class_counts)

# Identify majority and minority classes
majority_class = class_counts.idxmax()
minority_class = class_counts.idxmin()

# Calculate the number of samples needed for oversampling
samples_to_add = class_counts[majority_class] - class_counts[minority_class]

# Oversample the minority class
oversampled_samples = df[df['aitah_tag'] == minority_class].sample(
    n=samples_to_add, replace=True, random_state=42  # Set random_state for reproducibility
)

# Add the oversampled instances to the original DataFrame
df_balanced = pd.concat([df, oversampled_samples])

# Shuffle the resulting dataframe to ensure randomness
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(df_balanced['aitah_tag'].value_counts())


aitah_tag
0    501
1     46
Name: count, dtype: int64


ValueError: a must be greater than 0 unless no samples are taken

## **Text Processing**

In [14]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [15]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/rx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/rx/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Gets a list of english stop words
stop_words = stopwords.words('english')

In [17]:
def preprocess_data(reddit_post):
    reddit_post = str(reddit_post).lower()
    reddit_post = re.sub("[^a-zA-Z0-9\s]",'', reddit_post)

    temp_final =[]

    for word in reddit_post.split():
        if word =='' or '\r\n' in word or word in stop_words:
            None
        else:
            temp_final.append(word)

    return word_tokenize(' '.join(temp_final))

In [18]:
df['processed_selftext'] = df['selftext'].apply(preprocess_data)

In [19]:
df.head()

,title,selftext,aitah_tag,processed_selftext
0,AITA for not inviting my mom to my wedding be...,I (28M) am getting married in a few months to ...,0,"[28m, getting, married, months, fiance, sarah,..."
1,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0,"[ive, told, story, subreddits, im, gon, na, te..."
2,AITA -,throw away account so no one sees..\n\nMy \[24...,0,"[throw, away, account, one, sees, 24f, partner..."
3,AITA - my 20 year old flatmate asks me questio...,For context I’m 33; and my flatmate is 20.\nSh...,0,"[context, im, 33, flatmate, 20, asks, question..."
4,"AITA - my husband never let's me sleep in, so ...","As the title states, I wake my husband up ever...",0,"[title, states, wake, husband, every, morning,..."


In [20]:
df['aitah_tag'].value_counts()

aitah_tag
0    501
1     46
Name: count, dtype: int64

## **Model Development**

### **Count Vectorizer**

In [21]:
#Importing count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
#Creating a CountVectorizer model
df['selftext'] = df['processed_selftext']
bow_transformer= CountVectorizer(analyzer=preprocess_data).fit(df['selftext'])

In [23]:
#Print number of vocabulary/words
print(len(bow_transformer.vocabulary_))

10526


In [24]:
article_bow=bow_transformer.transform(df['selftext'])

### **TF-IDF**

In [25]:
#Import TFIDF
from sklearn.feature_extraction.text import TfidfTransformer
#Making an instance of this transformer
tfidf_transformer=TfidfTransformer().fit(article_bow)

In [26]:
#Checking the tfidf of a particular word('husband'), it'll directly error if it doesnt see the word
tfidf_transformer.idf_[bow_transformer.vocabulary_['husband']]

2.8995560396837625

In [27]:
#Convert the entire bag of words corpus into a tfidf corpus at once
article_tfidf=tfidf_transformer.transform(article_bow)

### **Support Vector Mac|hine**

In [28]:
#lets import SVM
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [29]:
selftext_train, selftext_test, aitah_tag_train, aitah_tag_test=train_test_split(df['selftext'],df['aitah_tag'],test_size=0.3)

In [30]:
#think of the pipeline as the steps or methods on a task
text_clf_svm = Pipeline([('vect', CountVectorizer(analyzer=preprocess_data)),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='log_loss', penalty='elasticnet',
                                            alpha=0.001, random_state=42, class_weight='balanced')),
 ])

In [31]:
_=text_clf_svm.fit(selftext_train, aitah_tag_train)

#### **Evaluation**

In [32]:
from sklearn.metrics import classification_report

In [33]:
prediction=text_clf_svm.predict(selftext_test)

In [34]:
# Check the class distribution in the training set
print("Training set class distribution:")
print(pd.Series(aitah_tag_train).value_counts())

# Check the class distribution in the test set
print("Test set class distribution:")
print(pd.Series(aitah_tag_test).value_counts())


Training set class distribution:
aitah_tag
0    344
1     38
Name: count, dtype: int64
Test set class distribution:
aitah_tag
0    157
1      8
Name: count, dtype: int64


In [35]:
# Generate a classification report
report = classification_report(aitah_tag_test, prediction, zero_division=0)

In [36]:
print(report)

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       157
           1       0.00      0.00      0.00         8

    accuracy                           0.95       165
   macro avg       0.48      0.50      0.49       165
weighted avg       0.91      0.95      0.93       165



#### **Testing**

In [37]:
# Example with manual mapping
label_mapping = {0: "NTA", 1: "YTA"}

# Predict the label
prediction_numeric = text_clf_svm.predict(["I gave her flowers"])[0]
prediction_label = label_mapping[prediction_numeric]

print(f"Predicted class: {prediction_label}")

Predicted class: NTA


### **FastText**

In [38]:
import fasttext

In [39]:
df2.head()

,title,selftext,aitah_tag
0,AITA for not inviting my mom to my wedding be...,I (28M) am getting married in a few months to ...,0
1,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0
2,AITA -,throw away account so no one sees..\n\nMy \[24...,0
3,AITA - my 20 year old flatmate asks me questio...,For context I’m 33; and my flatmate is 20.\nSh...,0
4,"AITA - my husband never let's me sleep in, so ...","As the title states, I wake my husband up ever...",0


In [40]:
# Split the training and testing dataset
selftext_train, selftext_test, aitah_tag_train, aitah_tag_test = train_test_split(df2['selftext'], df2['aitah_tag'], test_size=0.3)

In [41]:
# Prepare data for FastText
def format_for_fasttext(texts, labels, filepath):
    with open(filepath, 'w') as f:
        for text, label in zip(texts, labels):
            f.write(f"__label__{label} {text}\n")

In [42]:
# Save training and testing data
format_for_fasttext(selftext_train, aitah_tag_train, "train_fasttext.txt")
format_for_fasttext(selftext_test, aitah_tag_test, "test_fasttext.txt")

In [49]:
# Train the model
ft_model = fasttext.train_supervised(input="train_fasttext.txt", epoch=25, lr=1.0, wordNgrams=2, verbose=2)

# Save the trained model for later use
ft_model.save_model("fasttext_aitah_model.bin")

Read 0M words
Number of words:  16306
Number of labels: 2
Progress: 100.0% words/sec/thread: 2240219 lr:  0.000000 avg.loss:  0.291151 ETA:   0h 0m 0s100.0% words/sec/thread: 2241376 lr: -0.000415 avg.loss:  0.291151 ETA:   0h 0m 0s


> This is overfitting na

#### **Evaluation**

In [50]:
# Test the model
result = ft_model.test("test_fasttext.txt")

In [51]:
result

(173, 0.9017341040462428, 0.9017341040462428)

In [52]:
# Output results
print(f"Precision: {result[1]:.2f}")
print(f"Recall: {result[2]:.2f}")
print(f"Number of examples: {result[0]}")

Precision: 0.90
Recall: 0.90
Number of examples: 173


#### **Testing**

In [58]:
# Define a label map
label_map = {#
    "__label__0": "Not the Asshole",
    "__label__1": "The Asshole"
}

In [59]:
# Predict top-k labels (e.g., top 2)
predicted_labels, confidences = ft_model.predict("Am I the asshole for yelling at my neighbor?", k=2)

# Display all predictions
for label, confidence in zip(predicted_labels, confidences):
    human_readable_label = label_map[label]
    print(f"Predicted Label: {human_readable_label}, Confidence: {confidence:.2f}")


Predicted Label: Not the Asshole, Confidence: 0.98
Predicted Label: The Asshole, Confidence: 0.02
